# Installing kubeflow pipelines client

In [25]:
import sys,json, os
job_class = os.getenv("DKUBE_JOB_CLASS")
if not job_class:
    !{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --upgrade >> /dev/null

In [26]:
SLURM_CLUSTER="slurm111"
run_id = 0
project_id = os.environ.get("DKUBE_PROJECT_ID", "")
project_name = os.environ.get("DKUBE_PROJECT_NAME")

# Check DKube project settings

In [27]:
if not project_id:
    project_name = "clinical-reg"
    from dkube.sdk import *
    api = DkubeApi(token=os.getenv("DKUBE_USER_ACCESS_TOKEN"))
    try:
        project = DkubeProject(project_name)
        res = api.create_project(project)
        project_id = res["id"]
    except Exception as e:
        if e.reason.lower()=="conflict":
            project_id = api.get_project_id(project_name)
            print(f"Project {project_name} exists.. reusing")
        else:
            raise e

Project clinical-reg exists.. reusing


In [28]:
print(f"project name = {project_name}")
print(f"project id = {project_id}")

project name = clinical-reg
project id = hpjajn


# Import kfp pkgs

In [29]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client
import os

In [30]:
setup_component = '''
name: create_dkube_resource
description: |
    creates dkube resources required for pipeline.
metadata:
  annotations: {platform: 'Dkube'}
  labels: {stage: 'create_dkube_resource', logger: 'dkubepl', wfid: '{{workflow.uid}}', runid: '{{pod.name}}'}
inputs:
  - {name: token,      type: String,   optional: false,
    description: 'Required. Dkube authentication token.'}
  - {name: user,      type: String,   optional: false,
    description: 'Required. Dkube Logged in User name.'}
  - {name: project_id,      type: String,   optional: false,
    description: 'Required. Dkube Project name.'}
implementation:
  container:
    image: docker.io/ocdr/dkube-examples-setup:cli-reg
    command: ['python3', 'regressionsetup.py']
    args: [
      --auth_token, {inputValue: token},
      --user, {inputValue: user},
      --project_id, {inputValue: project_id}
    ]
'''

# Define e2e regression Pipeline with Dkube components

In [31]:
import kfp
import kfp.dsl as dsl
from kfp import components
from kfp.components._yaml_utils import load_yaml
from kfp.components._yaml_utils import dump_yaml
from kubernetes import client as k8s_client

import os
import json
from random import randint

from dkube.slurm.job import *
from dkube.sdk import DkubeTraining, DkubePreprocessing
from dkube.slurm.job_properties import JobProperties

def _component(stage, name):
    with open('/mnt/dkube/pipeline/components/{}/component.yaml'.format(stage), 'rb') as stream:
        cdict = load_yaml(stream)
        cdict['name'] = name
        cyaml = dump_yaml(cdict)
        return components.load_component_from_text(cyaml)
        
setup_op = kfp.components.load_component(text = setup_component)

@dsl.pipeline(
    name='dkube-regression-pl',
    description='sample regression pipeline with dkube components'
)

def d3pipeline(
    user,
    token,
    project_id=project_id,
    tags = json.dumps([]),
    #Slurm cluster name
    slurm_cluster = SLURM_CLUSTER,
    #Slurm cluster properties
    slurm_jobprops = JobProperties().to_dict(),
    
    
    #Clinical preprocess
    clinical_preprocess_script="python cli-pre-processing.py",
    clinical_preprocess_datasets=json.dumps(["clinical"]),
    clinical_preprocess_input_mounts=json.dumps(["/opt/dkube/input"]),
    clinical_preprocess_outputs=json.dumps(["clinical-preprocessed"]),
    clinical_preprocess_output_mounts=json.dumps(["/opt/dkube/output"]),
    
    #Image preprocess
    image_preprocess_script="python img-pre-processing.py",
    image_preprocess_datasets=json.dumps(["images"]),
    image_preprocess_input_mounts=json.dumps(["/opt/dkube/input"]),
    image_preprocess_outputs=json.dumps(["images-preprocessed"]),
    image_preprocess_output_mounts=json.dumps(["/opt/dkube/output"]),
    
    #Clinical split
    clinical_split_script="python split.py --datatype clinical",
    clinical_split_datasets=json.dumps(["clinical-preprocessed"]),
    clinical_split_input_mounts=json.dumps(["/opt/dkube/input"]),
    clinical_split_outputs=json.dumps(["clinical-train", "clinical-test", "clinical-val"]),
    clinical_split_output_mounts=json.dumps(["/opt/dkube/outputs/train", "/opt/dkube/outputs/test", "/opt/dkube/outputs/val"]),
    
    #Image split
    image_split_script="python split.py --datatype image",
    image_split_datasets=json.dumps(["images-preprocessed"]),
    image_split_input_mounts=json.dumps(["/opt/dkube/input"]),
    image_split_outputs=json.dumps(["images-train", "images-test", "images-val"]),
    image_split_output_mounts=json.dumps(["/opt/dkube/outputs/train", "/opt/dkube/outputs/test", "/opt/dkube/outputs/val"])	,
    
    #RNA split
    rna_split_script="python split.py --datatype rna",
    rna_split_datasets=json.dumps(["rna"]),
    rna_split_input_mounts=json.dumps(["/opt/dkube/input"]),
    rna_split_outputs=json.dumps(["rna-train", "rna-test", "rna-val"]),
    rna_split_output_mounts=json.dumps(["/opt/dkube/outputs/train", "/opt/dkube/outputs/test", "/opt/dkube/outputs/val"]),
    
    #Training
    job_group = 'default',
    #Framework. One of tensorflow, pytorch, sklearn
    framework = "tensorflow",
    #Framework version
    version = "2.3.0",
    #In notebook DKUBE_USER_ACCESS_TOKEN is automatically picked up from env variable
    #Or any other custom image name can be supplied.
    #For custom private images, please input username/password
    training_container=json.dumps({'image':'ocdr/dkube-datascience-tf-cpu:v2.3.0-6'}),
    #Name of the workspace in dkube. Update accordingly if different name is used while creating a workspace in dkube.
    training_program="regression",
    #Script to run inside the training container    
    training_script="python train_nn.py --epochs 5",
    #Input datasets for training. Update accordingly if different name is used while creating dataset in dkube.    
    training_datasets=json.dumps(["clinical-train", "clinical-val", "images-train",
                                  "images-val", "rna-train", "rna-val"]),
    training_input_dataset_mounts=json.dumps(["/opt/dkube/inputs/train/clinical", "/opt/dkube/inputs/val/clinical",
                                      "/opt/dkube/inputs/train/images", "/opt/dkube/inputs/val/images",
                                      "/opt/dkube/inputs/train/rna", "/opt/dkube/inputs/val/rna"]),
    training_outputs=json.dumps(["regression-model"]),
    training_output_mounts=json.dumps(["/opt/dkube/output"]),
    #Request gpus as needed. Val 0 means no gpu, then training_container=docker.io/ocdr/dkube-datascience-tf-cpu:v1.12    
    training_gpus=0,
    #Any envs to be passed to the training program    
    training_envs=json.dumps([{"steps": 100}]),
    
    tuning=json.dumps({}),
    
    #Evaluation
    evaluation_script="python evaluate.py",
    evaluation_datasets=json.dumps(["clinical-test", "images-test", "rna-test"]),
    evaluation_input_dataset_mounts=json.dumps(["/opt/dkube/inputs/test/clinical", "/opt/dkube/inputs/test/images",
                                      "/opt/dkube/inputs/test/rna"]),
    evaluation_models=json.dumps(["regression-model"]),
    evaluation_input_model_mounts=json.dumps(["/opt/dkube/inputs/model"]),
    
    #Serving
    #Device to be used for serving - dkube mnist example trained on gpu needs gpu for serving else set this param to 'cpu'
    serving_device='cpu',
    #Serving image
    serving_image=json.dumps({'image':'ocdr/tensorflowserver:2.3.0'}),
    #Transformer image
    transformer_image=json.dumps({'image':'ocdr/dkube-datascience-tf-cpu:v2.3.0-17'}),
    #Script to execute the transformer
    transformer_code="clinical_reg/transformer.py"):
    
    create_resource = setup_op(user = user, token = token, project_id = project_id).set_display_name("(dkube)create_resources")
    create_resource.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    
    clinical_preprocess_job = DkubePreprocessing(str(user), tags=tags)
    clinical_preprocess_job.update_container(image_url="ocdr/dkube-datascience-tf-cpu:v2.3.0-6")
    clinical_preprocess_job.update_startupscript(str(clinical_preprocess_script))
    clinical_preprocess_job.add_code(str(training_program))
    clinical_preprocess_job.add_input_dataset("clinical", mountpath="/opt/dkube/input")
    clinical_preprocess_job.add_output_dataset("clinical-preprocessed", mountpath="/opt/dkube/output")
    clinical_preprocess = dkube_slurmjob_preprocessing_op(
                        slurm_cluster,
                        slurm_jobprops,
                        str(user), 
                        str(token), 
                        clinical_preprocess_job.job.to_dict()).set_display_name("(slurm)clinical-preprocess").after(create_resource)
    
    
    image_preprocess_job = DkubePreprocessing(str(user), tags=tags)
    image_preprocess_job.update_container(image_url="ocdr/dkube-datascience-tf-cpu:v2.3.0-6")
    image_preprocess_job.update_startupscript(str(image_preprocess_script))
    image_preprocess_job.add_code(str(training_program))
    image_preprocess_job.add_input_dataset("images", mountpath="/opt/dkube/input")
    image_preprocess_job.add_output_dataset("images-preprocessed", mountpath="/opt/dkube/output")
    
    image_preprocess = dkube_slurmjob_preprocessing_op(
                        slurm_cluster,
                        slurm_jobprops,
                        str(user), 
                        str(token), 
                        image_preprocess_job.job.to_dict()).set_display_name("(slurm)images-preprocess").after(create_resource)
    
    clinical_split_job = DkubePreprocessing(str(user), tags=tags)
    clinical_split_job.update_container(image_url="ocdr/dkube-datascience-tf-cpu:v2.3.0-6")
    clinical_split_job.update_startupscript(str(clinical_split_script))
    clinical_split_job.add_code(str(training_program))
    clinical_split_job.add_input_dataset("clinical-preprocessed", mountpath="/opt/dkube/input")
    clinical_split_job.add_output_dataset("clinical-train", mountpath="/opt/dkube/outputs/train")
    clinical_split_job.add_output_dataset("clinical-test", mountpath="/opt/dkube/outputs/test")
    clinical_split_job.add_output_dataset("clinical-val", mountpath="/opt/dkube/outputs/val")
    
    clinical_split = dkube_slurmjob_preprocessing_op(
                    slurm_cluster,
                    slurm_jobprops,
                    str(user), 
                    str(token), 
                    clinical_split_job.job.to_dict()).set_display_name("(slurm)clinical-split").after(clinical_preprocess)
    
    

    image_split_job = DkubePreprocessing(str(user), tags=tags)
    image_split_job.update_container(image_url="ocdr/dkube-datascience-tf-cpu:v2.3.0-6")
    image_split_job.update_startupscript(str(image_split_script))
    image_split_job.add_code(str(training_program))
    image_split_job.add_input_dataset("images-preprocessed", mountpath="/opt/dkube/input")
    image_split_job.add_output_dataset("images-train", mountpath="/opt/dkube/outputs/train")
    image_split_job.add_output_dataset("images-test", mountpath="/opt/dkube/outputs/test")
    image_split_job.add_output_dataset("images-val", mountpath="/opt/dkube/outputs/val")
    
    image_split = dkube_slurmjob_preprocessing_op(
                    slurm_cluster,
                    slurm_jobprops,
                    str(user), 
                    str(token), 
                    image_split_job.job.to_dict()).set_display_name("(slurm)images-split").after(image_preprocess)
    
                                      
    rna_split  = _component('preprocess', 'rna-split')(container=training_container, tags=[tags],
                                      program=training_program, run_script=rna_split_script,
                                      datasets=rna_split_datasets, outputs=rna_split_outputs,
                                      input_dataset_mounts=rna_split_input_mounts, output_mounts=rna_split_output_mounts).after(create_resource)

    training_job = DkubeTraining(str(user), tags=tags)
    training_job.update_container(image_url="ocdr/dkube-datascience-tf-cpu:v2.3.0-6")
    training_job.update_startupscript(str(training_script))
    training_job.add_code(str(training_program))
    training_job.add_input_dataset("clinical-train", mountpath='/opt/dkube/inputs/train/clinical')
    training_job.add_input_dataset("clinical-val", mountpath='/opt/dkube/inputs/val/clinical')
    training_job.add_input_dataset("images-train", mountpath='/opt/dkube/inputs/train/images')
    training_job.add_input_dataset("images-val", mountpath='/opt/dkube/inputs/val/images')
    training_job.add_input_dataset("rna-train", mountpath='/opt/dkube/inputs/train/rna')
    training_job.add_input_dataset("rna-val", mountpath='/opt/dkube/inputs/val/rna')
    training_job.add_output_model("regression-model", mountpath='/opt/dkube/output')
    training_job.add_envvars({"steps": "100"})
    
    
    train = dkube_slurmjob_op(
                slurm_cluster,
                slurm_jobprops,
                str(user), 
                str(token), 
                training_job.job.to_dict()).set_display_name("(slurm) training").after(clinical_split).after(image_split).after(rna_split)
    

    serving     = _component('serving', 'model-serving')(model="regression-model", device=serving_device,
                                serving_image=serving_image,
                                transformer_image=transformer_image,
                                transformer_project=training_program,
                                transformer_code=transformer_code).set_display_name("(dkube) serving").after(train)
    inference   = _component('viewer', 'model-inference')(servingurl=serving.outputs['servingurl'],
                                 servingexample='regression', viewtype='inference').set_display_name("(dkube) viewer").after(serving)
    

# Compile and generate tar ball

In [32]:
import random, string
suffix = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(6))
pipeline_filename = 'dkube_regression_pl_full.tar.gz'
pipeline_name = 'Regression Pipeline' + "-" + suffix
compiler.Compiler().compile(d3pipeline, pipeline_filename)

## Upload Pipeline

In [33]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=existing_token, namespace=os.getenv("USERNAME"))
try:
  client.upload_pipeline(pipeline_package_path = pipeline_filename, pipeline_name = pipeline_name, description = None)
except BaseException as e:
    print(e)

In [34]:
if project_id:
    print(project_id)
    tags = f"project:{project_id}"
    run_name = f"[{project_name}] Clinical Reg {run_id}"
    experiment_name = f"[{project_name}] experiment"
else:
    print(project_id)
    tags = []
    run_name = f"Clinical Reg {run_id}"
    experiment_name = "Dkube - Clinical Reg"

hpjajn


## Create regression experiment

In [35]:
user = os.getenv("USER")
client.list_experiments(namespace=user)
# Create a new experiment
try:
    clinical_experiment = client.create_experiment(name=experiment_name, namespace=user)
except BaseException as e:
    print(e)

## Create a run

In [36]:
user = os.getenv("USER")
auth_token = existing_token

try:
    run = client.run_pipeline(clinical_experiment.id, run_name, pipeline_package_path=pipeline_filename, service_account=user,
                              params={"user":user, "token":auth_token, "tags": tags})
except BaseException as e:
    print(e)